In [8]:
%pylab inline
%load_ext rpy2.ipython

Populating the interactive namespace from numpy and matplotlib
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


### Practice #1
#### What are best options for remote synchronization using *rsync*?

### Prompts:
- what about subfolders?
- what about links in folders?
- what about unchanged access permitions on each file (owner is owner)?


* -a, --archive               archive mode; equals -rlptgoD (no -H,-A,-X)<br>
* -r, --recursive             recurse into directories<br>
* -l, --links                 copy symlinks as symlinks<br>
* -p, --perms                 preserve permissions<br>
* -t, --times                 preserve modification times<br>
* -g, --group                 preserve group<br>
     --devices               preserve device files (super-user only)<br>
     --specials              preserve special files<br>
* -D                          same as --devices --specials<br>
* -o, --owner                 preserve owner (super-user only)<br>


* -z, --compress              compress file data during the transfer
     --compress-level=NUM    explicitly set compression level   ex. 9
* --partial-dir=DIR       put a partially transferred file into DIR
* --partial               keep partially transferred files
* -h, --human-readable        output numbers in a human-readable format
* --no-o 
* --groupmap=*:GROUPONREMOTE 
* --chmod=Dg+s,ug+r,go-w,o-rwx to change permision

Incremental recursion can be disabled using the --no-inc-recursive
    option or its shorter --no-i-r alias. 



In [6]:
%%bash
echo rsync -a --compress --compress-level=9 \
--info=progress2 --no-i-r --human-readable \
--partial-dir=.rsync-partial --partial \
--perms --no-o --groupmap=*:group_name --chmod=Dg+s,ug+r,go-w,o-rwx \
    -e ssh /from/where/copy/files/ --rsync-path="mkdir -p \"${REMOTE_DIR}${OUTDIR}\" && rsync" "${REMOTE}${OUTDIR}/"

#for i in $(seq 10); do sleep 1; echo $i; done


rsync -a --compress --compress-level=9 --info=progress2 --no-i-r --human-readable --partial-dir=.rsync-partial --partial --perms --no-o --groupmap=*:group_name --chmod=Dg+s,ug+r,go-w,o-rwx -e ssh /from/where/copy/files/ --rsync-path=mkdir -p "" && rsync /
1
2
3
4
5
6
7
8
9
10


### Practise #2
#### What options need to be added to run rsync live... safely?

### Prompts:
- what about half-created files?
- when to finish coping?

- -partial-dir=DIR       put a partially transferred file into DIR
- -partial               keep partially transferred files


In [ ]:
%%bash
echo TODATE="$(date --date '+48 hours 20 minutes' +%s)" \
&& while [ $TODATE -ge "$(date +%s)" ]; do rsync_scipt.sh sample_name_dir; sleep 1200; done

#### *How to rsync only files in fastq format

## Example #1
Divide file into 3 equal files.

In [ ]:
#https://stackoverflow.com/questions/21034373/how-to-load-edit-run-save-text-files-py-into-an-ipython-notebook-cell

In [10]:
%%writefile example.fasta
>1
ACGT
>2
GCTA
>3
TTTT
>4
ACCC

Writing example.fasta


In [33]:
%%writefile Snakefile

rule divide_into_3:
    input: seq="{name}.fasta",
    output: seq1="{name}1.fasta",
            seq2="{name}2.fasta",
            seq3="{name}3.fasta",            
    shell: """
        awk '{{nr=int((NR-1)/2)%3+1; x="{wildcards.name}"nr".fasta";}}{{print > x}}' "{input.seq}"
    """ # (this string goes through .format(snake_variables_dictionary))
    

Overwriting Snakefile


In [34]:
%%bash
snakemake example3.fasta -f && tail -n +1 example{1,2,3}.fasta

==> example1.fasta <==
>1
ACGT
>4
ACCC

==> example2.fasta <==
>2
GCTA

==> example3.fasta <==
>3
TTTT


Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	divide_into_3
	1

[Tue Sep 18 12:25:32 2018]
rule divide_into_3:
    input: example.fasta
    output: example3.fasta, example2.fasta, example1.fasta
    jobid: 0
    wildcards: name=example

[Tue Sep 18 12:25:32 2018]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /mnt/zgmvol/_forge/wkolczynska/workspace/genXone/brca/report/src/.snakemake/log/2018-09-18T122532.051400.snakemake.log


### Practise #3

In [ ]:
%%writefile Snakefile
# TASK write 
# Warning, please generate strings (output list, shell) using + operator on strings, not .format function (ask for explanation why :)
n=5
rule divide_into_n:
    ...

In [39]:
%%writefile Snakefile
# TASK write 
# Warning, please generate strings (output list, shell) using + operator on strings, not .format function (ask for explanation why :)
n=4
rule divide_into_n:
    input: seq="{name}.fasta",
    output: ["{name}"+str(i)+".fasta" for i in range(1,n+1)],            
    shell: """
        awk '{{nr=int((NR-1)/2)%"""+str(n)+"""+1; x="{wildcards.name}"nr".fasta";}}{{print > x}}' "{input.seq}"
    """ # (this string goes through .format(snake_variables_dictionary))
    

Overwriting Snakefile


In [40]:
%%bash
snakemake example3.fasta -f && tail -n +1 example{1,2,3,4}.fasta

==> example1.fasta <==
>1
ACGT

==> example2.fasta <==
>2
GCTA

==> example3.fasta <==
>3
TTTT

==> example4.fasta <==
>4
ACCC


Building DAG of jobs...
Using shell: /bin/bash
Provided cores: 1
Rules claiming more threads will be scaled down.
Job counts:
	count	jobs
	1	divide_into_n
	1

[Tue Sep 18 12:28:09 2018]
rule divide_into_n:
    input: example.fasta
    output: example1.fasta, example2.fasta, example3.fasta, example4.fasta
    jobid: 0
    wildcards: name=example

[Tue Sep 18 12:28:09 2018]
Finished job 0.
1 of 1 steps (100%) done
Complete log: /mnt/zgmvol/_forge/wkolczynska/workspace/genXone/brca/report/src/.snakemake/log/2018-09-18T122809.736711.snakemake.log


## Example 2
Case when the number of outputs is unknown or large. If all downstream rules rely on the whole sets of outputs, rather than on the individual files, and when we want to create output for for example every sample we have, 'dynamic' keyword.
This rule will launch other rules which are necessary to create files. 


In [ ]:
rule all_centrifuge_aggregated:
    input:[dynamic("{{numer}}/{{number}}.from_centrifuge".format(suffix)) for suffix in ["noEuca__krona.html", "classification_krona.html"]]


## Practice 3
### Snakemake

Using snakemake create 3 rules that enable creation of:
1. ...150 random sequences in FASTA format (each from 50 to 100bp). Put them randomly in 3 files, so that each file contains 100 sequences (sequences can be repeated in files or even in one file). PYTHON lub bas (shuff)
2. ... another file (tab separated) containing any sequence that occurred in the files and at least: the number of different files in which it was placed, and the number of all counts. SHELL
3. ... interesting histograms based on 4th file (each sequence in the number of files). R

### Practise #4
Convert fastq file into fasta.

In [ ]:
%%bash
cat example.fastq | awk 'NR%4 == 1 {print ">" substr($0, 2)} NR%4 == 2 {print}'

In [ ]:
What about multilines fastq?

### Practice #5

Get randomly 10% of data from fasta file (2 lines == one record, don’t split it!). 

Pseudorandom != random

## BLAST

### Practice #7
What options of BLAST could help improved outputs for nanopore data?

paramset:
- -task blastn/blastp/tblastn/blastx
- -num_threads 
- -db
- -max_target_seqs  
- -evalue
- -word_size 
- -max_hsps  
- -outfmt
    - %f means sequence in FASTA format
    - %s means sequence data (without defline)
    - %l means sequence length
    - %T means taxid
    - %L means common taxonomic name

### Practice #8
#### RUN BLAST


0. Download database from:
ftp://ftp.ncbi.nlm.nih.gov/blast/db/16SMicrobial.tar.gz

1. Prepare sample:
fastq into fasta

2. Run Blast:
/mnt/zgmvol/environment/software/sources/genXone/ncbi-blast-2.7.1+-src/c++/ReleaseMT/bin/blastn   -num_threads 32    -evalue  1e3      -word_size 15    -max_hsps  1     -max_target_seqs 5
-db path_to_baza    -query input.fasta 
-outfmt "{params.outfmt}" > output.tsv

3. Filtr:
 "{params.script}" blastFilter "{input.datain}" "{params.dmp}" "{output.dataout}" {params.all_flag} -l "{output.datalog}"

4. Count 
datamash --sort -g 3 count 3 mean 4 median 4 mean 5 median 5 < "{input.datain}" | sort -k 2rn,2 -k 1 > "{output.dataout}"  #BLAST_OUTPUT='6 qseqid sseqid staxids bitscore length qlen mismatch score evalue'

5. from blast to krakn
 awk 'BEGIN{{ print "readID\tseqID\ttaxID\tscore\t2ndBestScore\thitLength\tqueryLength\tnumMatches"; FS="\t"; OFS="\t";}}{{print   $1, $2,$3, $8, 0, $5, $6, 1 }} ' "{input.datain}" > "{output.dataout}" && centrifuge-kreport -x "{params.db}" "{output.dataout}" > "{output.krakn}"


### Practice #9
1. Pavian

- Check how many Bacterial and Viral reads are in sample. 
- Create visualization in the form of a text and graphical tree. 
- Compare 2 samples in PAVIAN. Create CSV and TSV file with this comparison. 

2. Krona
- Compare results from KRONA and Pavian. Where are the defferences?
